The following is an intermediate use case. The first block here just defines some experimental constants.

In [10]:
import math
import pandas as pd

# Venturi Constants
Cd = 0.8828                                  # Venturi meter discharge coefficient
A1 = (0.622*0.0254)**2*math.pi/4             # Large Venturi Area
A2 = (0.228*0.0254)**2*math.pi/4             # Small Venturi Area

# HEMJ Constants
LTC = 0.0005                                 # Ideal Distance to Cooled Surface for TCs [m]
Dj = 0.00104                                 # Center Jet Diameter [m]
Dj2 = 0.0006                                 # Other Jet Diameter [m]
Dflat = 0.00118                              # Flat Jet Diameter (FR1H07) [m]
Di = 0.00954                                 # Inner Tube Diameter [m]
Aj = Dj**2*math.pi/4 + 24*Dj2**2*math.pi/4   # Total Jet Cross Sectional Area [m^2]
Ah = 0.0169926**2*math.pi/4;                 # Heated Surface Area [m^2]
Ac = 184.21e-6;
Ai = Di**2*math.pi/4;                        # Inner Tube Cross-sectional area [m^2]
P = Dj*math.pi + 24*Dj2*math.pi;             # Total Jet Perimeter [m^2]
Dh = 4*Aj/P;                                 # Hydraulic Diameter [m]

# Geometry Error
eLTC = 0.0001
LTC8=0.0008 
LTC6=0.00084 
LTC4=0.0008 
LTC2=0.0006        
eLTC8 = 0.2*LTC8 
eLTC6 = 0.2*LTC6 
eLTC4 = 0.2*LTC4 
eLTC2 = 0.2*LTC2
eDj = 0.00005
eDj2 = 0.00005
eDflat = 0.00005
eAj = math.pi/4*((2*Dj*eDj)**2+(2*Dj2*eDj2)**2)**(1/2)
eAh = math.pi/2*0.017*0.00005
eAc = 2*math.pi*0.00647*0.00005
eP = eDj*math.pi + 24*eDj2*math.pi
eDh = ((4/P*eAj)**2+(-4*Aj/P**2*eP)**2)**(1/2)

The next step is to read the data into a pandas DataFrame using the read_csv function

In [128]:
_data = pd.read_csv('legacy_code/Helium Loop 2_20_2018 11_39_43.csv',sep = '\t',
                  header = 0,skiprows = 50,parse_dates = True,index_col = 1,skip_blank_lines = True,encoding = 'utf-16')

columns = []
for c in _data.columns:
    _c = c.replace('<','').replace('(','[').replace(')',']').replace('>','').split(' ')
    try:
        columns.append(''.join(_c[1:]))
    except IndexError:
        columns.append(_c[0])

df = pd.DataFrame(_data.to_numpy(),index = _data.index,columns = columns)
print(df.shape)

(620, 43)
